In [1]:
import pandas as pd
import numpy as np

In [2]:
csv_path = "schools_complete.csv"
schools_complete = pd.read_csv(csv_path)
schools_complete.head(3)

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600


In [3]:
student_csv = "students_complete.csv"
student_complete = pd.read_csv(student_csv)
student_complete.head(3)

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60


In [4]:
total_schools = len(schools_complete)

In [5]:
total_students = sum(schools_complete["size"])

In [6]:
total_budget = sum(schools_complete["budget"])

In [7]:
average_math_score = student_complete["math_score"].mean()

In [8]:
average_reading_score = student_complete["reading_score"].mean()

In [9]:
pass_math = student_complete.loc[student_complete["math_score"] >= 60]
percent_pass_math = len(pass_math)/total_students

In [10]:
pass_reading = student_complete.loc[student_complete["reading_score"] >= 60]
percent_pass_reading = len(pass_reading)/total_students

In [11]:
overall_passing_rate = (percent_pass_math + percent_pass_reading)/2

In [12]:
district_summary_df = pd.DataFrame.from_records({"Total Schools in District": total_schools, "Number of Students in District": total_students, 
                 "Total District Budget": total_budget, "Average Math Scores for District": average_math_score, 
                 "Average Reading Score for District": average_reading_score, "Percent of Students Passing Math Testing": 
                  percent_pass_math, "Percent of Students Passing Reading Testing": percent_pass_reading, 
                  "Overall Pass Rate": overall_passing_rate}, index=[0])
district_summary = district_summary_df.iloc[:,::1]
district_summary_df

,Average Math Scores for District,Average Reading Score for District,Number of Students in District,Overall Pass Rate,Percent of Students Passing Math Testing,Percent of Students Passing Reading Testing,Total District Budget,Total Schools in District
0,78.985371,81.87784,39170,0.962229,0.924457,1.0,24649428,15


In [87]:
student_complete_new = student_complete.rename(columns = {"name":"student_name", "school" : "name"})
student_complete_new.head(3)

,Student ID,student_name,gender,grade,name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60


In [88]:
student_complete_new["reading_pass_fail"] = student_complete_new["reading_score"] >= 60
student_complete_new["math_pass_fail"] = student_complete_new["math_score"] >= 60
student_complete_new.head(3)

,Student ID,student_name,gender,grade,name,reading_score,math_score,reading_pass_fail,math_pass_fail
0,0,Paul Bradley,M,9th,Huang High School,66,79,True,True
1,1,Victor Smith,M,12th,Huang High School,94,61,True,True
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,True,True


In [89]:
school_test_totals = student_complete_new.pivot_table(index = ["name"], values = ["math_score", "reading_score"], 
                    aggfunc = np.sum) 
school_test_totals_df = school_test_totals.reset_index()
school_test_totals.head(3)

,math_score,reading_score
name,,
Bailey High School,383393,403225
Cabrera High School,154329,156027
Figueroa High School,226223,239335


In [16]:
schools_with_scores = pd.merge(schools_complete, school_test_totals_df, how = "outer", on = "name")
schools_with_scores.head(3)

,School ID,name,type,size,budget,math_score,reading_score
0,0,Huang High School,District,2917,1910635,223528,236810
1,1,Figueroa High School,District,2949,1884411,226223,239335
2,2,Shelton High School,Charter,1761,1056600,146796,147441


In [17]:
per_student_budget = schools_with_scores["budget"]/schools_with_scores["size"]

In [18]:
average_math_school = schools_with_scores["math_score"]/schools_with_scores["size"]

In [19]:
average_reading_school = schools_with_scores["reading_score"]/schools_with_scores["size"]

In [20]:
school_pass_fail = student_complete_new.pivot_table(index = ["name"], values = ["reading_pass_fail" , "math_pass_fail"]) 
school_pass_fail_df = school_pass_fail.reset_index()
school_pass_fail_df["school_overall_pass"] =  (school_pass_fail_df["math_pass_fail"] + school_pass_fail_df["reading_pass_fail"])/2

In [21]:
schools_with_scores["per_student_budget"] = per_student_budget
schools_with_scores["average_math_score"] = average_math_school
schools_with_scores["average_reading_score"] = average_reading_school
schools_with_scores.head(3)

,School ID,name,type,size,budget,math_score,reading_score,per_student_budget,average_math_score,average_reading_score
0,0,Huang High School,District,2917,1910635,223528,236810,655.0,76.629414,81.182722
1,1,Figueroa High School,District,2949,1884411,226223,239335,639.0,76.711767,81.158020
2,2,Shelton High School,Charter,1761,1056600,146796,147441,600.0,83.359455,83.725724


In [90]:
school_summary = pd.merge(schools_with_scores, school_pass_fail_df, how = "outer", on = "name")
school_summary.head(3)

,School ID,name,type,size,budget,math_score,reading_score,per_student_budget,average_math_score,average_reading_score,math_pass_fail,reading_pass_fail,school_overall_pass
0,0,Huang High School,District,2917,1910635,223528,236810,655.0,76.629414,81.182722,0.888584,1.0,0.944292
1,1,Figueroa High School,District,2949,1884411,226223,239335,639.0,76.711767,81.158020,0.884368,1.0,0.942184
2,2,Shelton High School,Charter,1761,1056600,146796,147441,600.0,83.359455,83.725724,1.000000,1.0,1.000000


In [23]:
school_district_summary = school_summary.drop(["math_score", "reading_score"], axis = 1)
school_district_summary

,School ID,name,type,size,budget,per_student_budget,average_math_score,average_reading_score,math_pass_fail,reading_pass_fail,school_overall_pass
0,0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.888584,1.0,0.944292
1,1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.884368,1.0,0.942184
2,2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,1.000000,1.0,1.000000
3,3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.890831,1.0,0.945415
4,4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,1.000000,1.0,1.000000
5,5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,1.000000,1.0,1.000000
6,6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,1.000000,1.0,1.000000
7,7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.895297,1.0,0.947649
8,8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,1.000000,1.0,1.000000
9,9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,1.000000,1.0,1.000000


In [24]:
schools_highest_overall = school_district_summary.nlargest(5, "school_overall_pass")
schools_highest_overall

,School ID,name,type,size,budget,per_student_budget,average_math_score,average_reading_score,math_pass_fail,reading_pass_fail,school_overall_pass
2,2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,1.0,1.0,1.0
4,4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,1.0,1.0,1.0
5,5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,1.0,1.0,1.0
6,6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,1.0,1.0,1.0
8,8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,1.0,1.0,1.0


In [25]:
schools_lowest_overall = school_district_summary.nsmallest(5, "school_overall_pass")
schools_lowest_overall

,School ID,name,type,size,budget,per_student_budget,average_math_score,average_reading_score,math_pass_fail,reading_pass_fail,school_overall_pass
1,1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.884368,1.0,0.942184
11,11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,0.885471,1.0,0.942736
0,0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.888584,1.0,0.944292
3,3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.890831,1.0,0.945415
12,12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.891829,1.0,0.945915


In [26]:
school_math_by_grade = student_complete_new.pivot_table(index = ["name"], values = ["math_score"], columns = "grade") 
school_math_by_grade = school_math_by_grade.reset_index()
school_math_by_grade

name math_score                                 
grade                              10th       11th       12th        9th
0         Bailey High School  76.996772  77.515588  76.492218  77.083676
1        Cabrera High School  83.154506  82.765560  83.277487  83.094697
2       Figueroa High School  76.539974  76.884344  77.151369  76.403037
3           Ford High School  77.672316  76.918058  76.179963  77.361345
4        Griffin High School  84.229064  83.842105  83.356164  82.044010
5      Hernandez High School  77.337408  77.136029  77.186567  77.438495
6         Holden High School  83.429825  85.000000  82.855422  83.787402
7          Huang High School  75.908735  76.446602  77.225641  77.027251
8        Johnson High School  76.691117  77.491653  76.863248  77.187857
9           Pena High School  83.372000  84.328125  84.121547  83.625455
10     Rodriguez High School  76.612500  76.395626  77.690748  76.859966
11       Shelton High School  82.917411  83.383495  83.778976  83.420755
12        Thomas High School  83.087886  83.498795  83.497041  83.590022
13        Wilson High School  83.724422  83.195326  83.035794  83.085578
14        Wright High School  84.010288  83.836782  83.644986  83.264706

In [27]:
school_reading_by_grade = student_complete_new.pivot_table(index = ["name"], values = ["reading_score"], 
                                                           columns = "grade") 
school_reading_by_grade = school_reading_by_grade.reset_index()
school_reading_by_grade

name reading_score                                 
grade                                 10th       11th       12th        9th
0         Bailey High School     80.907183  80.945643  80.912451  81.303155
1        Cabrera High School     84.253219  83.788382  84.287958  83.676136
2       Figueroa High School     81.408912  80.640339  81.384863  81.198598
3           Ford High School     81.262712  80.403642  80.662338  80.632653
4        Griffin High School     83.706897  84.288089  84.013699  83.369193
5      Hernandez High School     80.660147  81.396140  80.857143  80.866860
6         Holden High School     83.324561  83.815534  84.698795  83.677165
7          Huang High School     81.512386  81.417476  80.305983  81.290284
8        Johnson High School     80.773431  80.616027  81.227564  81.260714
9           Pena High School     83.612000  84.335938  84.591160  83.807273
10     Rodriguez High School     80.629808  80.864811  80.376426  80.993127
11       Shelton High School     83.441964  84.373786  82.781671  84.122642
12        Thomas High School     84.254157  83.585542  83.831361  83.728850
13        Wilson High School     84.021452  83.764608  84.317673  83.939778
14        Wright High School     83.812757  84.156322  84.073171  83.833333

In [86]:
bins = [550, 600, 650, 700]
bin_names_spending = ["Low", "Medium", "High"]
school_summary["summary spending per student"] = pd.cut(school_summary["per_student_budget"], bins, labels = bin_names_spending)
school_summary_spending = school_summary.drop(["School ID","type", "size", "budget", "math_score", "reading_score", 
                                               "per_student_budget", "size by number of students"], axis=1)
school_summary_spending

,name,average_math_score,average_reading_score,math_pass_fail,reading_pass_fail,school_overall_pass,summary spending per student
0,Huang High School,76.629414,81.182722,0.888584,1.0,0.944292,High
1,Figueroa High School,76.711767,81.158020,0.884368,1.0,0.942184,Medium
2,Shelton High School,83.359455,83.725724,1.000000,1.0,1.000000,Low
3,Hernandez High School,77.289752,80.934412,0.890831,1.0,0.945415,High
4,Griffin High School,83.351499,83.816757,1.000000,1.0,1.000000,Medium
5,Wilson High School,83.274201,83.989488,1.000000,1.0,1.000000,Low
6,Cabrera High School,83.061895,83.975780,1.000000,1.0,1.000000,Low
7,Bailey High School,77.048432,81.033963,0.895297,1.0,0.947649,Medium
8,Holden High School,83.803279,83.814988,1.000000,1.0,1.000000,Low
9,Pena High School,83.839917,84.044699,1.000000,1.0,1.000000,Medium


In [40]:
bins = [0, 1500, 3500, 5500]
bin_names_size = ["Small", "Medium", "Large"]
school_summary["size by number of students"] = pd.cut(school_summary["size"], bins, labels=bin_names_size)
school_summary_size = school_summary.drop(["School ID", "type", "size", "budget", "per_student_budget", "math_score",
                                          "reading_score", "summary spending per student"], axis=1)
school_summary_size

,name,average_math_score,average_reading_score,math_pass_fail,reading_pass_fail,school_overall_pass,size by number of students
0,Huang High School,76.629414,81.182722,0.888584,1.0,0.944292,Medium
1,Figueroa High School,76.711767,81.158020,0.884368,1.0,0.942184,Medium
2,Shelton High School,83.359455,83.725724,1.000000,1.0,1.000000,Medium
3,Hernandez High School,77.289752,80.934412,0.890831,1.0,0.945415,Large
4,Griffin High School,83.351499,83.816757,1.000000,1.0,1.000000,Small
5,Wilson High School,83.274201,83.989488,1.000000,1.0,1.000000,Medium
6,Cabrera High School,83.061895,83.975780,1.000000,1.0,1.000000,Medium
7,Bailey High School,77.048432,81.033963,0.895297,1.0,0.947649,Large
8,Holden High School,83.803279,83.814988,1.000000,1.0,1.000000,Small
9,Pena High School,83.839917,84.044699,1.000000,1.0,1.000000,Small


In [81]:
summary_chartered = school_summary[school_summary.type == "Charter"]
summary_chartered = summary_chartered.drop(["School ID", "size", "budget", "math_score", "reading_score", "per_student_budget"], axis = 1)
summary_chartered

,name,type,average_math_score,average_reading_score,math_pass_fail,reading_pass_fail,school_overall_pass,summary spending per student,size by number of students
2,Shelton High School,Charter,83.359455,83.725724,1.0,1.0,1.0,Low,Medium
4,Griffin High School,Charter,83.351499,83.816757,1.0,1.0,1.0,Medium,Small
5,Wilson High School,Charter,83.274201,83.989488,1.0,1.0,1.0,Low,Medium
6,Cabrera High School,Charter,83.061895,83.975780,1.0,1.0,1.0,Low,Medium
8,Holden High School,Charter,83.803279,83.814988,1.0,1.0,1.0,Low,Small
9,Pena High School,Charter,83.839917,84.044699,1.0,1.0,1.0,Medium,Small
10,Wright High School,Charter,83.682222,83.955000,1.0,1.0,1.0,Low,Medium
14,Thomas High School,Charter,83.418349,83.848930,1.0,1.0,1.0,Medium,Medium


In [83]:
summary_district = school_summary[school_summary.type == "District"]
summary_district = summary_district.drop(["School ID", "size", "budget", "math_score", "reading_score", "per_student_budget"], axis = 1)
summary_district

,name,type,average_math_score,average_reading_score,math_pass_fail,reading_pass_fail,school_overall_pass,summary spending per student,size by number of students
0,Huang High School,District,76.629414,81.182722,0.888584,1.0,0.944292,High,Medium
1,Figueroa High School,District,76.711767,81.158020,0.884368,1.0,0.942184,Medium,Medium
3,Hernandez High School,District,77.289752,80.934412,0.890831,1.0,0.945415,High,Large
7,Bailey High School,District,77.048432,81.033963,0.895297,1.0,0.947649,Medium,Large
11,Rodriguez High School,District,76.842711,80.744686,0.885471,1.0,0.942736,Medium,Large
12,Johnson High School,District,77.072464,80.966394,0.891829,1.0,0.945915,Medium,Large
13,Ford High School,District,77.102592,80.746258,0.893027,1.0,0.946513,Medium,Medium
